In [1]:
import re
import docx

In [2]:
def get_tests(table):
    units = {}
    data = {}
    referent_col_name =''
    rows_num = len(table.rows)
    cells_num = len(table.rows[1].cells)
    blank_rows_ind = []
    is_referent_col = False  #referent column flag ("ед.измер", "Норма")
    
    #get referent name ("ед.измер", "Норма")
    if table.rows[1].cells[1].text == 'ед.изм' or table.rows[1].cells[1].text == 'Норма':
        referent_col_name = table.rows[1].cells[1].text
        is_referent_col = True

    #get blank rows
    for i in range(rows_num):
        row = table.rows[i]
        if (row.cells[0].text == '' and row.cells[1].text == ''):
            blank_rows_ind.append(i)
    blank_rows_ind.append(len(table.rows))

    #get sample units
    if is_referent_col:
        for row in range(1, rows_num):
            unit_name = table.rows[row].cells[0].text
            if unit_name != '':
                units[table.rows[row].cells[0].text] = table.rows[row].cells[1].text
            else: 
                continue
    
    #get samples results
    start_row = 0
    start_col = 2 if is_referent_col else 1
    for blank in blank_rows_ind:
        for cell in range(start_col, cells_num):
            date = table.rows[start_row+1].cells[cell].text.strip()
            if date == '':
                continue
            test = {}
            for row in range (start_row+2, blank):
                observation = table.rows[row].cells[cell].text.strip()
                if observation != '':
                    observation_name = table.rows[row].cells[0].text.strip()
                    test[observation_name] = observation
                else: 
                    continue
            data[date] = test
        start_row = blank
    return{'referent_col_name': referent_col_name, 'units': units, 'data': data}

In [9]:
# doc = docx.Document('../data/103 Вишневский_1726_1818.docx')
doc = docx.Document('../data/0 123_3.docx')

In [11]:
for table in doc.tables:
    print (table.rows[0].cells[0].text)

ФИО
Общие анализы крови
Биохимические анализы крови
Общие анализы мочи
Иммунологические исследования
Дополнительные исследования
Микробиологические посевы
Этиотропная терапия
Температурный лист


In [12]:
tables_mapping = {'ФИО': 'general_info', 
                  'Общие анализы крови': 'cbc',
                  'Биохимические анализы крови': 'blood_tests',
                  'Общие анализы мочи': 'urine_tests',
                  'Иммунологические исследования': 'immunologic_tests',
                  'Дополнительные исследования': 'additional_tests',
                  'Микробиологические посевы': 'microbiology',
                  'Этиотропная терапия': 'antibiotics',
                  'Температурный лист': 'temperature'}

In [13]:
tables = {}
for table in doc.tables:
    tables[tables_mapping[table.rows[0].cells[0].text.strip()]] = table
tables

{'general_info': <docx.table.Table at 0xda2f30>,
 'cbc': <docx.table.Table at 0xda2f50>,
 'blood_tests': <docx.table.Table at 0xda2f90>,
 'urine_tests': <docx.table.Table at 0xda2fb0>,
 'immunologic_tests': <docx.table.Table at 0xda2fd0>,
 'additional_tests': <docx.table.Table at 0xda2970>,
 'microbiology': <docx.table.Table at 0xda28d0>,
 'antibiotics': <docx.table.Table at 0xda2890>,
 'temperature': <docx.table.Table at 0xda27b0>}

In [9]:
#get general info
gen_info_mapping = {'ФИО': 'name',
                    'Дата рождения': 'birthdate',
                    'Адрес домашний': 'address',
                    'Отделение': 'department',
                    'Дата госпитализации': 'admission_date',
                    'Дата выписки': 'discharge_date',
                    'Клинический диагноз': 'diagnosis'}
general_info = {}
the_table = tables['general_info']
for row in the_table.rows:
    general_info[gen_info_mapping[row.cells[0].text.strip()]] = row.cells[1].text
general_info

{'name': 'ЙЦУ',
 'birthdate': '11.11.1988',
 'address': 'ФЫВА.',
 'department': 'РАО -> 2 хирургическое',
 'admission_date': '04.09.2018',
 'discharge_date': '05.12.2018',
 'diagnosis': 'ЖКБ. Хронический калькулезный холецистит. Холедохолитиаз. Механическая желтуха. Острый некротический билиарный панкреатит. Инфицированный панкреонекроз. Гнойный оментобурсит. Парапанкреальная флегмона слева. Некроз брыжейки поперечно-ободочной кишки. Острая язва желудка, осложненная кровотечением от 11.09.2018г. Сепсис. Сепстический шок от 07.09.2018г. Левосторонний реактивный. Полифакторная анемия средней степени. Первичный гипотиреоз, ремиссия.'}

In [14]:
#get temperature data
temperature = {}
the_table = tables['temperature']
rows_num = len(the_table.rows)
cells_num = len(the_table.rows[1].cells)
temp_id = 0
for row in range(1, rows_num, 3):
    for cell in range(0, cells_num):
        date = the_table.rows[row].cells[cell].text
        if date != '':
            temperature[temp_id] = {'date':date,
                                    'temp': the_table.rows[row+1].cells[cell].text}
            temp_id+=1
        else: 
            break
temperature[0]

{'date': '04.09', 'temp': '36.8'}

In [15]:
#get Complete blood count
cbc = get_tests(tables['cbc'])

In [16]:
cbc

{'referent_col_name': 'ед.изм',
 'units': {'RBC': '10^12/л',
  'Hgb': 'г/л',
  'Hct': '',
  'Tr': '10^9/л',
  'WBC': '10^9/л',
  'юные': '%',
  'п/яд': '%',
  'с/яд': '%',
  'лимф': '%',
  'мон': '%',
  'эоз': '%',
  'СОЭ': 'мм/час',
  'ТЗН': '',
  'миел': '%',
  'баз': '%'},
 'data': {'04.09': {'RBC': '3.80',
   'Hgb': '161',
   'Hct': '44.3',
   'Tr': '281',
   'WBC': '22.17',
   'п/яд': '5',
   'с/яд': '81',
   'лимф': '10',
   'мон': '3',
   'эоз': '1',
   'СОЭ': '45'},
  '05.09': {'RBC': '4.10',
   'Hgb': '134',
   'Hct': '36.7',
   'Tr': '286',
   'WBC': '17.02',
   'п/яд': '5',
   'с/яд': '86',
   'лимф': '4',
   'мон': '5',
   'СОЭ': '8'},
  '06.09': {'RBC': '3.92',
   'Hgb': '127',
   'Hct': '35.9',
   'Tr': '271',
   'WBC': '16.18',
   'п/яд': '9',
   'с/яд': '77',
   'лимф': '3',
   'мон': '10',
   'эоз': '1',
   'СОЭ': '10',
   'ТЗН': '++'},
  '07.09': {'RBC': '3.42',
   'Hgb': '111',
   'Hct': '31.8',
   'Tr': '274',
   'WBC': '17.71',
   'п/яд': '6',
   'с/яд': '80',
   '

In [17]:
#get Blood tests
blood_tests = get_tests(tables['blood_tests'])

In [18]:
blood_tests

{'referent_col_name': 'ед.изм',
 'units': {'общ бел': 'г/л',
  'альбум': 'г/л',
  'амил': 'Е/л',
  'глюк': 'Мм/л',
  'мочев': 'ммоль/л',
  'креат': 'Мкм/л',
  'калий': 'Мм/л',
  'натрий': 'Мм/л',
  'хлориды': 'Мм/л',
  'кальций': 'Мм/л',
  'СРБ': 'мг/л',
  'билир общ': 'Мкм/л',
  'билир пр': 'Мкм/л',
  'АсАТ': 'Е/л',
  'АлАТ': 'Е/л',
  'АЧТВ': 'сек',
  'ПТИ': '',
  'фибриног': 'г/л',
  'Д-димеры': 'нг/мл',
  'холестерин': 'ммоль/л',
  'ЛПВП': 'ммоль/л',
  'ЛПНП': 'ммоль/л',
  'ТАГ': 'ммоль/л',
  'ПКТ': 'нг/мл'},
 'data': {'04.09': {'общ бел': '76',
   'амил': '173',
   'глюк': '7.8',
   'мочев': '2.2',
   'креат': '79',
   'калий': '6.1',
   'натрий': '136',
   'хлориды': '101',
   'кальций': '1.80',
   'билир общ': '162.6',
   'билир пр': '57.5',
   'АсАТ': '140',
   'АлАТ': '245',
   'АЧТВ': '23',
   'ПТИ': '0.81',
   'фибриног': '4.4'},
  '05.09': {'общ бел': '45',
   'амил': '88',
   'глюк': '10.5',
   'мочев': '2.2',
   'креат': '64',
   'калий': '3.32',
   'хлориды': '104',
   'к

In [19]:
#get Urine tests
urine_tests = get_tests(tables['urine_tests'])
# urine_tests = get_tests(doc.tables[3])

In [20]:
urine_tests

{'referent_col_name': '',
 'units': {},
 'data': {'04.09': {'цвет': 'н/жел',
   'мутн': 'мутн',
   'реакция': 'кисл',
   'отн. плотн': '1030',
   'белок': '6.5',
   'глюк': '2.8',
   'эп. плоск': '6-8',
   'эритр': '10-12',
   'лейк': '28-30',
   'амилаза (N до 653)': '5499'},
  '05.09': {'цвет': 'жел',
   'мутн': 'неп',
   'реакция': 'кисл',
   'отн. плотн': 'м/м',
   'белок': '0.98',
   'глюк': '64.2',
   'эп. плоск': '2-3',
   'эритр': '14-16',
   'лейк': '14-16',
   'амилаза (N до 653)': '1453'},
  '06.09': {'цвет': 'жел',
   'мутн': 'полн',
   'реакция': 'кисл',
   'отн. плотн': 'м/м',
   'белок': '0.23',
   'глюк': '45.1',
   'эп. плоск': '2-4',
   'эритр': '6-8',
   'лейк': '4-6'},
  '07.09': {'цвет': 'жел',
   'мутн': 'неп',
   'реакция': 'кисл',
   'отн. плотн': 'м/м',
   'белок': '0.57',
   'глюк': '39.7',
   'эп. плоск': '1-2',
   'эритр': 'бол кол-во',
   'лейк': '2-3'},
  '08.09': {'цвет': 'с/ж',
   'мутн': 'неп',
   'реакция': 'кисл',
   'отн. плотн': 'м/м',
   'белок': '

In [21]:
#get Immunologic tests
immunologic_tests = get_tests(tables['immunologic_tests'])

In [22]:
immunologic_tests

{'referent_col_name': 'Норма',
 'units': {'Т-лимфоциты (Е-РОК)': '58-67%',
  'Т-лимфоциты активные': '24-30%',
  'Т-хелперы CD-4': '35-48%',
  'T-супрессоры CD8': '18-25%',
  'Иммунорегуляторный индекс (ИРИ=Тх/Тс)': '1,4-2,0',
  'B-лимфоциты СD22': '16-24%',
  'Иммуноглобулины G': '8-18 г/л',
  'Иммуноглобулины А': '0,9-45 г/л',
  'Иммуноглобулины М': '0,6-2,5 г/л',
  'Иммунные комплексы': 'до 56 Ед',
  'Фагоцитарный индекс': '80-90%',
  'Фагоцитарное число': '8,9-12,3'},
 'data': {'12.09': {'Т-лимфоциты (Е-РОК)': '43',
   'Т-лимфоциты активные': '20',
   'Т-хелперы CD-4': '23',
   'T-супрессоры CD8': '20',
   'Иммунорегуляторный индекс (ИРИ=Тх/Тс)': '1.1',
   'B-лимфоциты СD22': '10',
   'Иммуноглобулины G': '6.9',
   'Иммуноглобулины А': '2.71',
   'Иммуноглобулины М': '1.36',
   'Иммунные комплексы': '175',
   'Фагоцитарный индекс': '36',
   'Фагоцитарное число': '5.5'},
  '24.09': {'Т-лимфоциты (Е-РОК)': '38',
   'Т-лимфоциты активные': '12',
   'Т-хелперы CD-4': '18',
   'T-супрес

In [80]:
# get antibiotics info
antibiotics = {}
table = tables['antibiotics']
rows_num = len(table.rows)
cells_num = len(table.rows[1].cells)

for row in range(1, rows_num):
    row_content = []
    for cell in range(cells_num):
        row_content.append(table.rows[row].cells[cell].text)
#     print(row_content) # ['с 04.09.2018 по 05.09.2018', 'Цефепим 2,0 * 2р. в/в']
    ab_name = row_content[1]
    ab_name = re.split("(\d)+", ab_name)[0]
    ab_dose = row_content[1].replace(ab_name, '')
    antibiotics[row-1] = {'name': ab_name.strip()}
    antibiotics[row-1]['dose'] = ab_dose
    antibiotics[row-1]['dates'] = []
    dates = row_content[0].split(',')
    for date in dates:
        d = date.strip().split(' ')
        antibiotics[row-1]['dates'].append({'from': d[1][:10], 'to':d[3][:10]})
antibiotics

{0: {'antibiotic': 'Цефепим',
  'dose': '2,0 * 2р. в/в',
  'dates': [{'from': '04.09.2018', 'to': '05.09.2018'}]},
 1: {'antibiotic': 'Левофлоксацин',
  'dose': '1,5 * 1р. в/в',
  'dates': [{'from': '04.09.2018', 'to': '13.09.2018'}]},
 2: {'antibiotic': 'Меропенем',
  'dose': '1,0 * 3р. в/в',
  'dates': [{'from': '06.09.2018', 'to': '13.09.2018'}]},
 3: {'antibiotic': 'Дорипенем',
  'dose': '1,0 в/в',
  'dates': [{'from': '14.09.2018', 'to': '19.09.2018'}]},
 4: {'antibiotic': 'Моксифлоксацин',
  'dose': '0,4 в/в',
  'dates': [{'from': '14.09.2018', 'to': '19.09.2018'},
   {'from': '05.10.2018', 'to': '19.10.2018'}]},
 5: {'antibiotic': 'Цефосульбактам',
  'dose': '2 фл. * 3р. в/в',
  'dates': [{'from': '18.09.2018', 'to': '20.09.2018'},
   {'from': '01.11.2018', 'to': '22.11.2018'}]},
 6: {'antibiotic': 'Сультасин',
  'dose': '2 фл. * 4р. в/в',
  'dates': [{'from': '20.09.2018', 'to': '27.09.2018'}]},
 7: {'antibiotic': 'Колистин',
  'dose': '4,5 млн ЕД * 2р. в/в',
  'dates': [{'from

In [17]:
import re
s = 'Левофлоксацин 1,5 * 12р. в/в'
str2 = re.split("(\d)+", s)
str2

['Левофлоксацин ', '1', ',', '5', ' * ', '2', 'р. в/в']

In [23]:
# get additional_tests info
additional_info = []
table = tables['additional_tests']
rows_num = len(table.rows)

for r_num in range(1, rows_num):
    test = {}
    test['date'] = table.rows[r_num].cells[0].text
    test['result'] = table.rows[r_num].cells[1].text
    additional_info.append(test)
print(additional_info)

[{'date': '04.09.2018', 'result': 'РТГ ОГК. Заключение: не исключаются следы выпота. Усиление, деформация легочного рисунка. Корни тяжисты. Срединная тень не смещена.'}, {'date': '04.09.2018', 'result': 'РТГ ОГК. Заключение: гидроторакс слева.'}, {'date': '04.09.2018', 'result': 'УЗИ ОБП. Заключение: острый панкреатит, панкреонекроз. Асцит. ЖКБ. Хронический калькулезный холецистит, обострение. Дилятация холедоха. Левосторонний гидроторакс. Диффузные изменения паренхимы почек.'}, {'date': '05.09.2018', 'result': 'ВЭГДС. Заключение: поверхностный гастрит. Дуодено-гастральный рефлюкс.'}, {'date': '05.09.2018', 'result': 'РКТ ОБП. Заключение: КТ-признаки деструктивного панкреатита, парапанкреатита. ЖКБ. Хронический калькулезный холецистит. Гепатомегалия, нерезко выраженная. Асцит. Двухсторонний плевральный выпот. Двухсторонняя нижнедолевая пневмония.'}, {'date': '05.09.2018', 'result': 'Кровь ИФА на ВИЧ. Результат: отрицательный.'}, {'date': '06.09.2018', 'result': 'Кровь ИФА на гепатиты. 